In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import math




             
yr=3.154e7
π=np.pi
c=3e8                  #in units of m s^-1
t_eq=1.59246e12        # in units of seconds
ρ_eq=2.15368e-16       # in units of kg.m^-3
G=6.67e-11             # in units of m^3.kg^-1.s^-2          
a_eq=2.9374e-4         # scale factor corresponding to matter radiation equlaity
t_i=13.78e9*yr         # in seconds
M_solar=1.989e30       # in units of kg


M_1PBH=1e-3*M_solar     # in units of kg
M_2PBH=10*M_solar      # in units of kg




x=np.geomspace(1e15,2e15,20)        #in units of m
x_list=x.tolist()


λ_list=np.zeros(len(x_list))

def λ_0(M_1PBH,M_2PBH,x): #λ_0 for the PBH binary
     return ((8*π*ρ_eq*(x**3))/(3*(M_1PBH+M_2PBH)))

for k, x in enumerate (x_list):
     λ_list[k]=λ_0(M_1PBH,M_2PBH,x) 

        




def M_halo(M_PBH,s):               #eq.(4) in reference pdf "M_DM at s_dec".
    return (((8*π*G*ρ_eq*(t_eq**(2)))/3)**(3/4))*(2**(5/4))*M_PBH* math.sqrt((((2/3)*(s-2)
                                *math.sqrt(s+1))+(4/3)))

def s_dec(λ_0):
    return (λ_0*s)

def h(x):
    return math.sqrt((x**-3)+(x**-4))   
def g(x):
    return  -((3*x+4)/(2*h(x)*(x**5))) #g(x)=h'(x).
def n(s):
    return (1+((M_halo(M_1PBH,s)+M_halo(M_2PBH,s))/(M_1PBH+M_2PBH)))
                   
def  k(s):
    return (1/n(s))


def λ(s):
    return λ_0*k(s)
   






def s_(λ_0):
    def f(u,s,λ_0):
            return [u[1], -((s*g(s)+h(s))*((s*u[1]-u[0]))/(h(s)*s**2))-((u[0])/((((h(s)*s)**2))*λ(s)
                     *((u[0]**2+b**2)**(3/2))))]    #eq.(1) in reference pdf "M_DM at s_dec".
    separation0= [1e-12,1]
    b= (3*λ_0)/100000
    ss= np.linspace(1e-12,((6*λ_0)/3),100000) 
    us= odeint(f,separation0,ss,args=( λ_0,))
    separations= us[:,0]
    As=ss/λ_0
    Bs=separations/λ_0
    
    plt.xlabel('s/λ_0')
    plt.ylabel('χ/λ_0')
    Bs_list = Bs.tolist()
    As_list = As.tolist()
    for i in  range(1,len(Bs_list)):
        Δ=(Bs[i] - Bs[i-1])
        s=As[i]
        if Δ<0:
            break
        
    return s
    
    
s_list=np.zeros(len(λ_list))
s_dec_list=np.zeros(len(λ_list))
M_halo_1_list = np.zeros(len(λ_list))
M_halo_2_list = np.zeros(len(λ_list))



for i, λ_0 in enumerate(λ_list):
    s_list[i] = s_(λ_0)

for i, (λ_0,s) in enumerate(zip(λ_list,s_list)):
    s_dec_list[i]= s_dec(λ_0)
    
for i,s in enumerate(s_dec_list):
    M_halo_1_list[i]=M_halo(M_1PBH,s)
    
for i,s in enumerate(s_dec_list):
    M_halo_2_list[i]=M_halo(M_2PBH,s)   
    





def a(λ_0):   #Function to calculate the initial semi-major axis of the PBH binary
    def f(u,s,λ_0):
        return [u[1], -((s*g(s)+h(s))*((s*u[1]-u[0]))/(h(s)*s**2))-((u[0])/((((h(s)*s)**2))*λ(s)
                     *((u[0]**2+b**2)**(3/2))))]    
    separation0= [1e-12,1]
    b=(3*λ_0)/100000
    ss= np.linspace(1e-12,((6*λ_0)/3),1000000) 
    us= odeint(f,separation0,ss,args=( λ_0,))
    separations= us[:,0]
    As=ss/λ_0
    Bs=separations/λ_0
    Bs_list = Bs.tolist()
    As_list = As.tolist()
    for i in  range(1,len(Bs_list)):
        Δ=(Bs[i] - Bs[i-1])
        A=Bs[i]
        a=(((A/2)*λ_0))                   #as Bs=separations/λ_0.
        if Δ<0:
            
            break
    
    
    return a

def semi(λ_0,x):
    return (a(λ_0)*x)


a_i_list=np.zeros(len(λ_list))
for i, (λ_0,x) in enumerate (zip(λ_list,x_list)):
    a_i_list[i] = semi(λ_0,x)
print("a_i=",a_i_list,"m")



def M_(M_PBH,M_DM):
    return (M_PBH+M_DM)
            

    
M_1_list=np.zeros(len(λ_list))
M_2_list=np.zeros(len(λ_list))

for i, M_DM in enumerate (M_halo_1_list):
    M_1_list[i] = M_(M_1PBH,M_DM)
    
for i, M_DM in enumerate (M_halo_2_list):
    M_2_list[i]=M_(M_2PBH,M_DM)





def j_i(M_1,M_2,a_i):
    return  ((((85/3)*(G**3)*M_1*M_2*(M_1+M_2)*t_i)/((a_i**4)*(c**5)))**(1/7))

j_i_list=np.zeros(len(M_1_list))


for k, (M_1,M_2,a_i) in enumerate (zip(M_1_list,M_2_list,a_i_list)):
    j_i_list[k]=j_i(M_1,M_2,a_i)
    

            

    
def e_i(j_i):
    return ((1-(j_i**2))**(1/2))

e_i_list=np.zeros(len(j_i_list))

for k, j_i in enumerate(j_i_list):
    e_i_list[k]=e_i(j_i)
    

print("e_i=",e_i_list)


def r_min(e_i,a_i): #r_min of the PBH binary orbit
    return (a_i*(1-e_i)) 

r_min_list=np.zeros(len(a_i_list))
for k, (e_i,a_i) in enumerate (zip(e_i_list,a_i_list)):
     r_min_list[k]=r_min(e_i,a_i)






def t_dec(s_dec):  #time of the binary decouple
    return (((3/(8*π*G*ρ_eq))**(1/2))*((2/3)*(s_dec-2)*((s_dec+1)**(1/2))+(4/3)))

def r_ta(M_PBH,s_dec): #r_turnaround of the dark matter halos
    return ((4*G*M_PBH*(t_dec(s_dec)**2))**(1/3))

r_ta_1_list=np.zeros(len(λ_list))
r_ta_2_list=np.zeros(len(λ_list))

for j, s_dec in enumerate (s_dec_list):
    r_ta_1_list[j]=r_ta(M_1PBH,s_dec)
    r_ta_2_list[j]=r_ta(M_2PBH,s_dec)
    
   



def ρ_bar(M_PBH):
    return ((ρ_eq/2)*(t_eq**(3/2))*((2*G*M_PBH)**(3/4))) 

def E_1(M_PBH,r_ta,r_min):
    return (16*π*G*ρ_bar(M_PBH)*M_PBH*(((r_ta)**(-1/4)))-((r_min/2)**(-1/4)))

def E_2(M_PBH,r_ta,r_min):
    return  (((128/3)*G*(π**2)*(ρ_bar(M_PBH)**2))*(((r_ta)**(1/2))-((r_min/2)**(1/2))))

def E_b(E_1,E_2):#Binding  energy of the dark matter halo
    return ((E_1-E_2))






E_1_1PBH_list=np.zeros(len(r_min_list))
E_2_1PBH_list=np.zeros(len(r_min_list))
E_1_2PBH_list=np.zeros(len(r_min_list))
E_2_2PBH_list=np.zeros(len(r_min_list))

for i, (r_ta,r_min) in enumerate (zip(r_ta_1_list,r_min_list)):
    E_1_1PBH_list[i] = E_1(M_1PBH,r_ta,r_min)
for i, (r_ta,r_min) in enumerate (zip(r_ta_1_list,r_min_list)):
    E_1_2PBH_list[i] = E_1(M_2PBH,r_ta,r_min)
    



for i, (r_ta,r_min) in enumerate (zip(r_ta_1_list,r_min_list)):
    E_2_1PBH_list[i] = E_2(M_1PBH,r_ta,r_min)
for i, (r_ta,r_min) in enumerate (zip(r_ta_2_list,r_min_list)):    
    E_2_2PBH_list[i] = E_2(M_2PBH,r_ta,r_min)

    

    
E_b_1PBH_list=np.zeros(len(r_min_list))
E_b_2PBH_list=np.zeros(len(r_min_list))

for i, (E_1,E_2) in enumerate (zip(E_1_1PBH_list,E_2_1PBH_list)):
    E_b_1PBH_list[i]=E_b(E_1,E_2)
    
for i, (E_1,E_2) in enumerate (zip(E_1_2PBH_list,E_2_2PBH_list)):
    E_b_2PBH_list[i]=E_b(E_1,E_2)
    




        
def M_halo(M_PBH,r): #Mass of the dark matter halos of size r<r_ta
    return ((16/3)*π*ρ_bar(M_PBH)*((r/2)**(3/4)))

M_halo_1f_list=np.zeros(len(λ_list))
M_halo_2f_list=np.zeros(len(λ_list))


for i, r in enumerate (r_min_list):
    M_halo_1f_list[i] = M_halo(M_1PBH,r)
    
for i, r in enumerate (r_min_list):
    M_halo_2f_list[i] = M_halo(M_2PBH,r)
    




def M_(M_PBH,M_halo):
    return (M_PBH+M_halo)


M_1f_list=np.zeros(len(λ_list))
M_2f_list=np.zeros(len(λ_list))

for i, M_halo in enumerate (M_halo_1f_list):
    M_1f_list[i] =  M_(M_1PBH,M_halo)
    
for i, M_halo in enumerate (M_halo_2f_list):
    M_2f_list[i] =  M_(M_2PBH,M_halo)
    
    


def a_f(a_i,M_1i,M_2i,M_1f,M_2f,E_b_1,E_b_2): #Final semi-major axis
    return ((G*M_1f*M_2f*a_i)/((G*M_1i*M_2i)-(2*a_i*(E_b_1+E_b_2))))


a_f_list=np.zeros(len(r_min_list))

for i, (a_i,M_1i,M_2i,M_1f,M_2f,E_b_1,E_b_2) in enumerate (zip(a_i_list,M_1_list,M_2_list,M_1f_list,M_2f_list,E_b_1PBH_list,E_b_2PBH_list)):
    a_f_list[i]=a_f(a_i,M_1i,M_2i,M_1f,M_2f,E_b_1,E_b_2)
    

print("a_f=",a_f_list,"m")
plt.loglog(a_i_list,a_f_list,'')